<a href="https://colab.research.google.com/github/adammoss/MLiS2/blob/reinforcement_learning/examples/reinforcement_learning/tabular/value_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this new algorithm, we will need to define two new objects: the value table, and the algorithm itself. We begin by pasting in the classes from the previous notebook, for reuse and comparison with the first algorithm.

In [ ]:
#@title
import math
import numpy as np
import numpy.random

class excursions(object):
	"""A simple environment which provides rewards based on excursions."""

	def __init__(self, parameters):
		self.trajectory_length = parameters['trajectory_length']
		self.positivity_bias = parameters['positivity_bias']
		self.target_bias = parameters['target_bias']
		self.action = 0
		self.state = [0, 0]
		self.terminal_state = False

	def _reward(self):
		"""Calculates the reward for the last transition to occur."""
		if self.state[0] < 0:
			reward = -self.positivity_bias * abs(self.state[0])
		else:
			reward = 0
		if self.state[1] == self.trajectory_length:
			reward -= self.target_bias * abs(self.state[0])
		return reward

	def step(self, action):
		"""Updates the environment state based on the input action."""
		self.action = action
		self.state[0] += 2*action - 1
		self.state[1] += 1
		if self.state[1] == self.trajectory_length:
			self.terminal_state = True
		return self.state, self._reward(), self.terminal_state

	def reset(self):
		"""Resets the environment state and terminal boolean."""
		self.action = 0
		self.state = [0, 0]
		self.terminal_state = False


class two_action_policy_table(object):
	"""A tabular policy for environments where each state has two actions."""

	def __init__(self, dimensions, learning_rate):
		self.table = np.zeros(dimensions)
		self.learning_rate = learning_rate

	def _forward(self, state):
		"""Calculates the probabilitiy of action 1."""
		exponentiated_potential = math.exp(-self.table[state[0]][state[1]])
		return 1/(exponentiated_potential+1)

	def action(self, state):
		"""Returns a random action according to the current policy."""
		action1_probability = self._forward(state)
		random = numpy.random.random()
		if random < action1_probability:
			return 1, 1 - action1_probability
		else:
			return 0, -action1_probability
	
	def step(self, state, error, eligibility):
		"""Updates the potential for actions in the given state."""
		self.table[state[0]][state[1]] += self.learning_rate * error * eligibility


class episodic_algorithm(object):
	"""A wrapper for episodic RL algorithms."""

	def __init__(self, parameters):
		self.environment = parameters['environment']
		self.average_return = 0
		self.average_returns = []
		self.returns = []
		self.return_learning_rate = parameters['return_learning_rate']
		self.policy = parameters['policy']
		self.episode = 0
		self.past_state = self.environment.state.copy()
		self.action = 0
		self.current_state = self.environment.state.copy()
		self.reward = 0
		self.current_return = 0
		self.terminal_state = False
	
	def _transition(self):
		"""Requests an action from the policy and sends it to the environment."""
		self.past_state = self.current_state.copy()
		self.action, self.eligibility = self.policy.action(self.current_state)
		self.current_state, self.reward, self.terminal_state = self.environment.step(
			self.action)
		self.current_return += self.reward

	def _per_step(self):
		"""A placeholder for a learning algorithms computations per transition."""
		self._transition()

	def _per_episode(self):
		"""A placeholder for a learning algorithms computations after episodes."""
		self.environment.reset()
		self.past_state = self.environment.state.copy()
		self.current_state = self.environment.state.copy()
		self.terminal_state = False

	def _episode(self):
		"""Uses _per_step and _per_episode to run a generic episodes computations."""
		self.current_return = 0
		while not self.terminal_state:
			self._per_step()
		self._per_episode()
		self.average_return += self.return_learning_rate * (self.current_return 
															- self.average_return)
		self.episode += 1

	def train(self, episodes):
		"""Trains the policy by repeatedly running episodes, storing return info."""
		self.episode = 0
		while self.episode < episodes:
			self._episode()
			self.average_returns.append(self.average_return)
			self.returns.append(self.current_return)

	def _sample(self):
		"""Generates a sample trajectory using the current policy."""
		self.current_return = 0
		trajectory = [self.current_state.copy()]
		while not self.terminal_state:
			self._transition()
			trajectory.append(self.current_state.copy())
		self.environment.reset()
		self.past_state = self.environment.state.copy()
		self.current_state = self.environment.state.copy()
		self.terminal_state = False
		return trajectory

	def samples(self, sample_count):
		"""Generates a set of trajectory samples."""
		trajectories = []
		sample = 0
		while sample < sample_count:
			trajectory = self._sample()
			trajectories.append(trajectory)
			sample += 1
		return trajectories

	def _return_sample(self):
		"""Runs an episode to sample a return for evaulation."""
		self.current_return = 0
		while not self.terminal_state:
			self._transition()
		self.environment.reset()
		self.past_state = self.environment.state.copy()
		self.current_state = self.environment.state.copy()
		self.terminal_state = False

	def evaluate(self, sample_count, set_average = True):
		"""Evaluates the policy by estimating the average return."""
		sample = 1
		average_return = 0
		while sample <= sample_count:
			self._return_sample()
			average_return += (self.current_return - average_return)/sample
			sample += 1
		if set_average:
			self.average_return = average_return
		return average_return


class monte_carlo_returns(episodic_algorithm):
	"""A purely return based policy gradient algorithm."""

	def __init__(self, parameters):
		super().__init__(parameters)
		self.states = []
		self.rewards = []
		self.eligibilities = []

	def _per_step(self):
		"""Adds required data storage for learning post-episode."""
		self._transition()
		self.states.append(self.past_state)
		self.rewards.append(self.reward)
		self.eligibilities.append(self.eligibility)

	def _update(self):
		"""Loops over the episode in reverse, updating the policy in each state."""
		self.rewards = np.array(self.rewards)
		state_return = 0
		for index in range(len(self.states) - 1, -1, -1):
			state_return += self.rewards[index]
			self.policy.step(self.states[index], state_return, self.eligibilities[index])

	def _per_episode(self):
		"""Adds additional resets relevant to learning algorithm."""
		self._update()
		super()._per_episode()
		self.states = []
		self.rewards = []
		self.eligibilities = []

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.1 64-bit ('gym': conda).

For the value table, we define essentially a stripped down version of the policy table: here, forward is called directly to access the value of each state.

In [ ]:
class value_table(object):
	"""A generic tabular state value function."""

	def __init__(self, dimensions, learning_rate):
		self.table = np.zeros(dimensions)
		self.learning_rate = learning_rate

	def forward(self, state):
		"""Returns the value of the specified state."""
		return self.table[state[0]][state[1]]

	def step(self, state, error):
		"""Updates the value of the specified state."""
		self.table[state[0]][state[1]] += self.learning_rate * error

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.1 64-bit ('gym': conda).

Our new algorithm is only a slight modification of the previous one, and as such it makes sense to inherit much of the behaviour of that class. The resulting class adds additional code for storing values of states along each episode, and replaces the update function so that this is used to in updates for both the values and policies of states along the episode.

In [ ]:
class monte_carlo_value_baseline(monte_carlo_returns):
	"""Contrasts returns with estimated values for policy updates."""

	def __init__(self, parameters):
		super().__init__(parameters)
		self.values = parameters['values']
		self.state_values = []

	def _per_step(self):
		"""Adds required data storage for learning post-episode."""
		super()._per_step()
		self.state_values.append(self.values.forward(self.past_state))

	def _update(self):
		"""Loops over the episode in reverse, updating state policies and values."""
		self.rewards = np.array(self.rewards)
		state_return = 0
		for index in range(len(self.states) - 1, -1, -1):
			state_return += self.rewards[index]
			error = state_return - self.state_values[index]
			self.policy.step(self.states[index], error, self.eligibilities[index])
			self.values.step(self.states[index], error)

	def _per_episode(self):
		"""Adds additional resets relevant to learning algorithm."""
		super()._per_episode()
		self.state_values = []

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.1 64-bit ('gym': conda).

For testing, we again initialize the environment, tables, and two agents using our two current algorithms.

In [ ]:
environment_parameters = dict(
	trajectory_length = 20, 
	positivity_bias = 1,
	target_bias = 2,
)
environment = excursions(environment_parameters)

table_dimension = (environment_parameters['trajectory_length']*2 + 1, 
				   environment_parameters['trajectory_length'] + 1)
policy1 = two_action_policy_table(table_dimension, 0.15)
values2 = value_table(table_dimension, 0.6)
policy2 = two_action_policy_table(table_dimension, 0.15)

algorithm_parameters1 = dict(
	environment = environment, 
	return_learning_rate = 0.1,
	policy = policy1,
)
algorithm_parameters2 = dict(
	environment = environment, 
	return_learning_rate = 0.1,
	values = values2,
	policy = policy2,
)
agent1 = monte_carlo_returns(algorithm_parameters1)
agent2 = monte_carlo_value_baseline(algorithm_parameters2)

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.1 64-bit ('gym': conda).

Next, we again evaluate the initial policy and store samples for comparison.

In [ ]:
from matplotlib import pyplot as plt

initial_return = agent1.evaluate(1000)
agent2.average_return = initial_return
print("Initial return: %s"%(initial_return))
initial_samples = agent1.samples(30)

min_y = np.min(np.array(initial_samples)[:,:,0]) - 1
max_y = np.max(np.array(initial_samples)[:,:,0]) + 1

plt.plot(np.array(initial_samples)[:,:,0].T, c = 'k', alpha = 0.2)
plt.scatter([20], [0], c = 'k', marker = 'o', s = 100)
plt.plot([-1, 21], [0, 0], lw = 2, c = 'r', ls = '--', alpha = 0.3)
plt.fill_between([-1, 21], [0, 0], [min_y, min_y], color = 'r', alpha = 0.1)
plt.xlim(-1, 21)
plt.ylim(min_y, max_y)
plt.xlabel("Time")
plt.ylabel("Position")
plt.show()

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.1 64-bit ('gym': conda).

Then, we train both agents using their respective algorithms, plotting their returns against eachother.

In [ ]:
episodes = 1000
agent1.train(episodes)
agent2.train(episodes)

plt.figure(figsize = (10, 3.5))
plt.subplot(121)
plt.plot(agent1.returns, c = 'b')
plt.plot(agent2.returns, c = 'm')
plt.xlabel("Episode")
plt.ylabel("Episodic returns")
plt.subplot(122)
plt.plot(agent1.average_returns, c = 'b')
plt.plot(agent2.average_returns, c = 'm')
plt.xlabel("Episode")
plt.ylabel("Running return")
plt.show()

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.1 64-bit ('gym': conda).

Finally, we compare both of these with the original policy.

In [ ]:
final_return1 = agent1.evaluate(1000)
final_return2 = agent2.evaluate(1000)
print("Initial return: %s, agent1's final return: %s, agent2's final return: %s"
%(initial_return, final_return1, final_return2))
samples1 = agent1.samples(30)
samples2 = agent2.samples(30)

plt.figure(figsize = (8, 3.5))
plt.subplot(121)
plt.plot(np.array(initial_samples)[:,:,0].T, c = 'k', alpha = 0.2)
plt.plot(np.array(samples1)[:,:,0].T, c = 'b', alpha = 0.2)
plt.scatter([20], [0], c = 'k', marker = 'o', s = 80)
plt.plot([-1, 21], [0, 0], lw = 2, c = 'r', ls = '--', alpha = 0.3)
plt.fill_between([-1, 21], [0, 0], [min_y, min_y], color = 'r', alpha = 0.1)
plt.xlim(-1, 21)
plt.ylim(min_y, max_y)
plt.title("Agent 1")
plt.xlabel("Time")
plt.ylabel("Position")
plt.subplot(122)
plt.plot(np.array(initial_samples)[:,:,0].T, c = 'k', alpha = 0.2)
plt.plot(np.array(samples2)[:,:,0].T, c = 'm', alpha = 0.2)
plt.scatter([20], [0], c = 'k', marker = 'o', s = 80)
plt.plot([-1, 21], [0, 0], lw = 2, c = 'r', ls = '--', alpha = 0.3)
plt.fill_between([-1, 21], [0, 0], [min_y, min_y], color = 'r', alpha = 0.1)
plt.xlim(-1, 21)
plt.ylim(min_y, max_y)
plt.title("Agent 2")
plt.xlabel("Time")
plt.show()

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.8.1 64-bit ('gym': conda).